In [ ]:
pip install unsloth transformers trl

In [ ]:
from datasets import load_dataset
import torch
from unsloth import FastLanguageModel
from trl import DPOTrainer
from unsloth.chat_templates import get_chat_template
from unsloth import PatchDPOTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True
 )

==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model, r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

Unsloth 2025.7.3 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

In [ ]:
dataset = load_dataset("argilla/ultrafeedback-binarized-preferences", split="train")

README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00001-9dffc9d46d32c335.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/63619 [00:00<?, ? examples/s]

In [ ]:
def convert_ultrafeedback(example):
    return {
        "prompt": example["instruction"],  # use plain string instead of chat format
        "chosen": example["chosen_response"],
        "rejected": example["rejected_response"]
    }
converted_dataset = dataset.map(convert_ultrafeedback, remove_columns=dataset.column_names)

Map:   0%|          | 0/63619 [00:00<?, ? examples/s]

In [ ]:
PatchDPOTrainer()
from trl import DPOConfig

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    output_dir="./dpo_outputs",
    max_length=2048
)

In [ ]:
trainer = DPOTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = converted_dataset,
    args = training_args
    )

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/63619 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/63619 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/63619 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 63,619 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhipodila1 (abhipodila1-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-421.076477,-302.854858,0.023867,-0.169876,0,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-208.453888,-183.823441,0.199251,-0.308899,No Log,No Log,No Log,No Log
3,0.691800,0.005024,0.002270,0.500000,0.002753,-304.741669,-233.733017,0.484396,-0.333470,No Log,No Log,No Log,No Log
4,0.683100,0.030069,0.009710,0.500000,0.020359,-405.741211,-287.802948,0.455936,0.067422,No Log,No Log,No Log,No Log
5,0.695800,0.030098,0.035012,0.500000,-0.004915,-292.809601,-195.757660,-0.036723,-0.478118,No Log,No Log,No Log,No Log
6,0.644100,0.100610,-0.001227,1.000000,0.101836,-395.565491,-274.173584,0.265678,-0.148797,No Log,No Log,No Log,No Log
7,0.726100,0.021824,0.083879,0.250000,-0.062055,-90.644829,-241.908646,-0.444501,-0.248757,No Log,No Log,No Log,No Log
8,0.612100,0.189854,0.003366,0.750000,0.186488,-353.592102,-258.298584,-0.529835,-0.740609,No Log,No Log,No Log,No Log
9,0.685400,0.011063,-0.004779,0.500000,0.015842,-130.710388,-106.773132,-0.408020,-0.806691,No Log,No Log,No Log,No Log
10,0.589200,0.224666,-0.005175,1.000000,0.229841,-446.353699,-273.958405,0.309044,0.064828,No Log,No Log,No Log,No Log


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=60, training_loss=0.6363836695750554, metrics={'train_runtime': 412.9255, 'train_samples_per_second': 0.581, 'train_steps_per_second': 0.145, 'total_flos': 0.0, 'train_loss': 0.6363836695750554, 'epoch': 0.003772457913516402})

In [ ]:
model.save_pretrained("fine_tuned_DPO_model")

In [ ]:
!zip -r all_files.zip /content

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.07.10/ (stored 0%)
  adding: content/.config/logs/2025.07.10/13.34.37.820504.log (deflated 58%)
  adding: content/.config/logs/2025.07.10/13.33.45.486303.log (deflated 92%)
  adding: content/.config/logs/2025.07.10/13.34.38.537881.log (deflated 57%)
  adding: content/.config/logs/2025.07.10/13.34.23.548658.log (deflated 86%)
  adding: content/.config/logs/2025.07.10/13.34.14.671755.log (deflated 58%)
  adding: content/.config/logs/2025.07.10/13.34.29.179991.log (deflated 58%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config

In [ ]:
model.eval()

In [ ]:
import torch

def generate_response(user_input):
    # Create chat messages
    messages = [
        {"role": "user", "content": user_input}
    ]

    # Format prompt using chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize and move to CUDA
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )

    # Decode and clean response
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded_output.split("Assistant:")[-1].strip()

    return response


In [ ]:
print(generate_response("Design a conversation between a customer and a customer service agent."))

system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

Design a conversation between a customer and a customer service agent.assistant

Here's a sample conversation between a customer and a customer service agent:

**Customer**: Hi, I'm calling about my recent order from your online store. I ordered a pair of shoes that I received, but they're not the color I ordered.

**Customer Service Agent**: I apologize for the inconvenience, and I'm happy to help you with that issue. Can you please provide me with your order number so I can look into this further?

**Customer**: It's #123456. I ordered the black boots, but the ones I got are brown.

**Customer Service Agent**: Thank you for providing the order number. I've located your order in our system, and I apologize for the mistake. We're going to go ahead and process a return for you. Can you please tell me your preferred method of return? Do you want us to provide a pre-paid return shipping label or would you like t